In [ ]:
'''
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
'''

In [ ]:
import math
import os
import shutil
import sys
from keras.preprocessing.image import array_to_img
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as ptc
import matplotlib
import seaborn as sns
import glob
import pydicom
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
'''
listfile=[]
files=os.listdir("../input/dicom-to-512jpg-train") 
for file in files: 
    if os.path.isfile("../input/dicom-to-512jpg-train/" + file):
        listfile.append(file)
len(listfile)
'''

In [ ]:
'''
#load data from dicom
my_annots=pd.read_csv("../input/dicom-to-jpg/2000_data_dim_convert.csv")
my_annots=my_annots.iloc[:,1:]
my_annots.image_id= my_annots.image_id.apply(lambda x: '../../input/dicom-to-jpg/'+  x + '.jpg')
annotations=my_annots[['image_id','x_min', 'y_min', 'x_max', 'y_max', 'class_name']]
annotations.to_csv('annotations.csv')
classes=my_annots.class_name.unique()
class_id=my_annots.class_id.unique()


with open('classes.csv', mode='w') as file:
    for  clss ,i in zip(classes,class_id ):
        file.write('{},{}\n'.format(clss, i))
amount_80 = int(0.8*len(annotations))
train_data = annotations[:amount_80]
test_data = annotations[amount_80:]
print(len(train_data))
print(len(test_data))

train_data.to_csv(path_or_buf='train_annotations.csv', index=False, header=None)
test_data.to_csv(path_or_buf='val_annotations.csv', index=False, header=None)

'''



In [ ]:
%ls

In [ ]:

train_data=pd.read_csv("/kaggle/input/img-dim-train/train_img_dim.csv")
annots=train_data.copy()
annots= annots[annots.class_id != 14] #drop NO Finding class
IMG_SIZE=512
def norm_df(df):
    df.x_min= ((df.x_min/ df.width)* IMG_SIZE).astype('int32')
    df.x_max=( (df.x_max/ df.width)* IMG_SIZE).astype('int32')
    
    df.y_min= ((df.y_min/ df.height)* IMG_SIZE).astype('int32')
    df.y_max= ((df.y_max/ df.height)* IMG_SIZE).astype('int32')
    return df
alldata=norm_df(annots.copy()).reset_index(drop = True)
#smalldata = df[0:20000] # first 20000 
#smalldata.to_csv('20000_data_dim_convert.csv')

In [ ]:
from sklearn.model_selection import StratifiedKFold
from matplotlib.pyplot import figure
import warnings

#load data from dicom
#my_annots=pd.read_csv("../../input/dicom-to-512jpg-train/20000_data_dim_convert.csv")
#my_annots=pd.read_csv("/kaggle/input/dicom-to-512jpg-train/20000_data_dim_convert.csv")
my_annots=alldata
#my_annots=my_annots.iloc[:,1:]
my_annots.image_id= my_annots.image_id.apply(lambda x: '../input/dicom-to-512jpg-train/'+  x + '.jpg')
annotations=my_annots[['image_id','x_min', 'y_min', 'x_max', 'y_max', 'class_name']]
annotations.to_csv('annotations.csv')
classes=my_annots.class_name.unique()
class_id=my_annots.class_id.unique()

with open('classes.csv', mode='w') as file:
    for  clss ,i in zip(classes,class_id ):
        file.write('{},{}\n'.format(clss, i))
#amount_80 = int(0.85*len(annotations))
#train_data = annotations[:amount_80]
#test_data = annotations[amount_80:]


warnings.filterwarnings('ignore')

#df = pd.read_csv(r'../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
df = pd.DataFrame(my_annots)
df = df[df['class_name'] != 'No finding']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)
df_folds = df[['image_id']].copy()

df_folds.loc[:, 'bbox_count'] = 1
df_folds = df_folds.groupby('image_id').count()
df_folds.loc[:, 'object_count'] = df.groupby('image_id')['class_id'].nunique()

df_folds.loc[:, 'stratify_group'] = np.char.add(
    df_folds['object_count'].values.astype(str),
    df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
)

df_folds.loc[:, 'fold'] = 0
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
    df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

# example with fold 0
df_folds.reset_index(inplace=True)

df_valid = pd.merge(df, df_folds[df_folds['fold'] == 0], on='image_id')

df_train = pd.merge(df, df_folds[df_folds['fold'] != 0], on='image_id')

figure(num=None, figsize=(30, 8))
df_train['class_name'].hist()
df_valid['class_name'].hist()
plt.show()

'''
train_data,test_data =train_test_split (annotations,test_size=0.15, random_state=42)
print(len(train_data))
print(len(test_data))

'''


In [ ]:

train_data=df_train[['image_id','x_min', 'y_min', 'x_max', 'y_max', 'class_name']]
test_data=df_valid[['image_id','x_min', 'y_min', 'x_max', 'y_max', 'class_name']]

In [ ]:
#train_data= train_data.drop(train_data[train_data.y_max== train_data.y_min].index[0])
test_data= test_data.drop(test_data[test_data.y_max== test_data.y_min].index[0])
train_data.to_csv(path_or_buf='train_annotations.csv', index=False, header=None)
test_data.to_csv(path_or_buf='val_annotations.csv', index=False, header=None)

In [ ]:
'''
'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5',
'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet101_oid_v1.0.0.h5',
'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet152_oid_v1.0.0.h5'
'''

In [ ]:

!mkdir weights
!wget -O /kaggle/working/weights/resnet50_coco_best_v2.h5 https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5

In [ ]:
!mkdir /kaggle/working/snapshots
!mkdir /kaggle/working/tensorboard

In [ ]:
!cd ~
!git clone https://github.com/fizyr/keras-retinanet

In [ ]:
#!git clone 'https://github.com/fizyr/keras-retinanet.git'
%cd keras-retinanet/
!pip install .
!python setup.py build_ext --inplace




In [ ]:
train_data.shape

In [ ]:
!cd ~
%ls

In [ ]:
cd /kaggle/working/keras-retinanet

In [ ]:
!keras_retinanet/bin/train.py --freeze-backbone \
  --random-transform \
  --workers 0 \
  --weights /kaggle/working/weights/resnet50_coco_best_v2.h5 \
  --batch-size 10 \
  --steps 352 \
  --image-min-side 512\
  --image-max-side 512\
  --epochs 50 \
  csv /kaggle/working/train_annotations.csv /kaggle/working/classes.csv


In [ ]:
from glob import glob
model_paths = glob('./snapshots/resnet50_csv_*.h5')
latest_path = sorted(model_paths)[-1]
print("path:", latest_path)



    


In [ ]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

#
#model = models.load_model(latest_path, backbone_name='resnet50')
#model = models.convert_model(model)

In [ ]:
!retinanet-convert-model ./snapshots/resnet50_csv_50.h5 \
/kaggle/working/weights/resnet50_csv_final.h5

In [ ]:
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
#from keras_retinanet.keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import csv
import os
import numpy as np
import time

In [ ]:
#%cd ./keras-retinanet
%ls


In [ ]:
model_path = os.path.join('/kaggle/working/weights', 'resnet50_csv_final.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')


##########

labels_to_names = {
    0: 'Aortic enlargement', 
    1: 'Atelectasis',
    2: 'Calcification',
    3: 'Cardiomegaly',
    4: 'Consolidation',
    5: 'ILD',
    6: 'Infiltration',
    7: 'Lung Opacity',
    8: 'Nodule/Mass',
    9:'Other lesion',
    10: 'Pleural effusion',
    11: 'Pleural thickening',
    12: 'Pneumothorax',
    13:'Pulmonary fibrosis'}


In [ ]:

%cd /kaggle/


In [ ]:
sub1=pd.read_csv('./input/sample/sample_submission .csv')
sub2=pd.read_csv('./input/test-img-dim/test.csv')
sub=pd.DataFrame.merge(sub1,sub2,on='image_id')
mysub=sub.copy()
mysub['thispath']='./input/dicom-to-512jpg-test/'+mysub.image_id+'.jpg'
mysub['score']=''
mysub.head()
#mysub['predictionstring']='14 1 0 0 1 1'



## predict and submission

In [ ]:
thr=0.5
#mysub=mysub[:50].copy() 
fields = ['image_id', 'PredictionString','score'] 
with open('submission.csv', mode='w') as csvfile:
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(fields)

    for row in tqdm(mysub.values) :
        imgpath=row[4]
        width=row[2]
        height=row[3]
        #predictionstring=row[4]
        
        image = read_image_bgr(imgpath)
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        
        x= int(width)/512
        y= int(height)/512
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            row[5]=score
            if score < thr:
                #row[1]="14 1 0 0 1 1"
                csvfile.write('{},{},{}\n'.format(row[0], row[1] ,row[5]))
                break
           
            row[1]= '{} {:.2f} {} {} {} {} '.format(label ,score , int(box[0]*x), int(box[1]*y),int((box[2])*x), int((box[3])*y))
            
            csvfile.write('{},{},{}\n'.format(row[0], row[1],row[5]))
presubmission=pd.read_csv('submission.csv')



In [ ]:
presubmission.to_csv('/kaggle/working/presubmission.csv',index = False)

In [ ]:
submission=presubmission.groupby('image_id')['PredictionString'].apply(' '.join).reset_index()

In [ ]:
submission.to_csv('/kaggle/working/submission.csv',index = False)



In [ ]:
!retinanet-evaluate csv /kaggle/working/val_annotations.csv /kaggle/working/classes.csv /kaggle/working/weights/resnet50_csv_final.h5

